In [ ]:
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])
decoded_review = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in train_data[0]])

In [ ]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [ ]:
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

**1. Cut off reviews after 150 words**

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
cutoff_x_train = pad_sequences(x_train, maxlen=150, truncating='post', padding='post')
cutoff_x_val = pad_sequences(x_val, maxlen=150, truncating='post', padding='post')
cutoff_x_test = pad_sequences(x_test, maxlen=150, truncating='post', padding='post')

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=10000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
model = get_model(150)
model.summary()


history = model.fit(
    cutoff_x_train,
    y_train,
    epochs=20,
    batch_size=512,
    validation_data=(cutoff_x_val, y_val)
)

test_loss, test_acc = model.evaluate(cutoff_x_test, y_test)
print(f"Test accuracy: {test_acc}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150)]             0         
                                                                 
 dense (Dense)               (None, 16)                2416      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2433 (9.50 KB)
Trainable params: 2433 (9.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
49/49 [==============================] - 3s 8ms/step - loss: 0.6989 - accuracy: 0.5508 - val_loss: 0.6556 - val_accuracy: 0.6296
Epoch 2/20
49/49 [===

**2. Restrict training samples to 100**

In [ ]:
model = get_model()
model.summary()


history = model.fit(
    partial_x_train[:100],
    partial_y_train[:100],
    epochs=20,
    batch_size=512,
    validation_data=(x_val, y_val)
)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                160016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160033 (625.13 KB)
Trainable params: 160033 (625.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.6996 - accuracy: 0.4900 - val_loss: 0.6958 - val_accuracy: 0.4982
Epoch 2/20
1/1

**3. Validate Only 10,000 Samples**

In [ ]:
model = get_model()
model.summary()


history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_val[:10000], y_val[:10000])
)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 10000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                160016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160033 (625.13 KB)
Trainable params: 160033 (625.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
30/30 [==============================] - 2s 45ms/step - loss: 0.5762 - accuracy: 0.7140 - val_loss: 0.4563 - val_accuracy: 0.8537
Epoch 2/20

**4. Consider only the top 10,000 words**

In [ ]:
model = get_model()
model.summary()


history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_val, y_val)
)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 10000)]           0         
                                                                 
 dense_6 (Dense)             (None, 16)                160016    
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160033 (625.13 KB)
Trainable params: 160033 (625.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
30/30 [==============================] - 2s 41ms/step - loss: 0.5824 - accuracy: 0.7133 - val_loss: 0.4694 - val_accuracy: 0.8437
Epoch 2/20

**5. Embedded Layer vs Pretrained Word Embedding**

In [ ]:
from tensorflow.keras.layers import Embedding
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=10000, output_dim=16)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 bidirectional (Bidirection  (None, 64)                12544     
 al)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 172609 (674.25 KB)
Trainable params: 172609 (674.25 KB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [ ]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_val, y_val)
)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

Epoch 1/20
30/30 [==============================] - 26s 750ms/step - loss: 0.6933 - accuracy: 0.4984 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 2/20
30/30 [==============================] - 21s 704ms/step - loss: 0.6933 - accuracy: 0.4989 - val_loss: 0.6937 - val_accuracy: 0.4947
Epoch 3/20
30/30 [==============================] - 21s 704ms/step - loss: 0.6932 - accuracy: 0.5053 - val_loss: 0.6932 - val_accuracy: 0.4958
Epoch 4/20
30/30 [==============================] - 21s 704ms/step - loss: 0.6932 - accuracy: 0.5021 - val_loss: 0.6931 - val_accuracy: 0.5207
Epoch 5/20
30/30 [==============================] - 21s 704ms/step - loss: 0.6931 - accuracy: 0.4995 - val_loss: 0.6933 - val_accuracy: 0.4947
Epoch 6/20
30/30 [==============================] - 21s 703ms/step - loss: 0.6931 - accuracy: 0.5052 - val_loss: 0.6931 - val_accuracy: 0.5054
Epoch 7/20
30/30 [==============================] - 21s 704ms/step - loss: 0.6933 - accuracy: 0.4979 - val_loss: 0.6933 - val_accuracy: 0.4947

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2024-04-19 23:42:31--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-04-19 23:42:31--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-04-19 23:42:32--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [ ]:
from tensorflow.keras.layers import TextVectorization

max_length = 200
max_tokens = 10000

embedding_dim = 100

text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=max_length,
    output_mode="int"
)

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

In [ ]:
inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 100)         1000000   
                                                                 
 bidirectional_6 (Bidirecti  (None, 64)                34048     
 onal)                                                           
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1034113 (3.94 MB)
Trainable params: 34113 (133.25 KB)
Non-trainable params: 1000000 (3.81 MB)
________________

In [ ]:
history = model.fit(
    partial_x_train[:500],
    partial_y_train[:500],
    epochs=20,
    batch_size=512,
    validation_data=(x_val, y_val)
)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

Epoch 1/20
1/1 [==============================] - 22s 22s/step - loss: 0.6908 - accuracy: 0.5340 - val_loss: 0.6863 - val_accuracy: 0.5800
Epoch 2/20
1/1 [==============================] - 22s 22s/step - loss: 0.6909 - accuracy: 0.5340 - val_loss: 0.6861 - val_accuracy: 0.5800
Epoch 3/20
1/1 [==============================] - 22s 22s/step - loss: 0.6909 - accuracy: 0.5340 - val_loss: 0.6860 - val_accuracy: 0.5800
Epoch 4/20
1/1 [==============================] - 22s 22s/step - loss: 0.6901 - accuracy: 0.5340 - val_loss: 0.6858 - val_accuracy: 0.5800
Epoch 5/20
1/1 [==============================] - 22s 22s/step - loss: 0.6918 - accuracy: 0.5340 - val_loss: 0.6858 - val_accuracy: 0.5800
Epoch 6/20
1/1 [==============================] - 23s 23s/step - loss: 0.6919 - accuracy: 0.5340 - val_loss: 0.6858 - val_accuracy: 0.5800
Epoch 7/20
1/1 [==============================] - 23s 23s/step - loss: 0.6911 - accuracy: 0.5340 - val_loss: 0.6858 - val_accuracy: 0.5800
Epoch 8/20
1/1 [===========

In [ ]:
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=max_length,
    output_mode="int"
)
train_texts = [" ".join([reverse_word_index.get(i - 3, '?') for i in review]) for review in train_data[:10000]]
x_val_texts = [" ".join([reverse_word_index.get(i - 3, '?') for i in review]) for review in x_val[:10000]]

train_texts = np.array(train_texts).reshape(-1)
x_val_texts = np.array(x_val_texts).reshape(-1)

text_vectorization.adapt(train_texts)

In [ ]:
train_texts.shape

(10000,)

In [ ]:
model = keras.Sequential([
    text_vectorization,
    embedding_layer,
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_15 (Tex  (None, 200)               0         
 tVectorization)                                                 
                                                                 
 embedding_6 (Embedding)     multiple                  1000000   
                                                                 
 bidirectional_12 (Bidirect  (None, 64)                34048     
 ional)                                                          
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1034113 (3.94 MB)
Trainable params: 34113 (133.25 KB)
Non-trainable params: 1000000 (3.81 MB)
_________________________________________________________________


In [ ]:
model.fit(
    train_texts,
    train_labels[:10000],
    epochs=20,
    batch_size=32,
    validation_data=(x_val_texts, y_val[:10000])
)

Epoch 1/20
313/313 [==============================] - 56s 151ms/step - loss: 0.6909 - accuracy: 0.5364 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 2/20
313/313 [==============================] - 43s 137ms/step - loss: 0.6731 - accuracy: 0.5810 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 3/20
313/313 [==============================] - 43s 136ms/step - loss: 0.6551 - accuracy: 0.6089 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 4/20
313/313 [==============================] - 44s 139ms/step - loss: 0.6331 - accuracy: 0.6427 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 5/20
313/313 [==============================] - 43s 138ms/step - loss: 0.6088 - accuracy: 0.6620 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 6/20
313/313 [==============================] - 43s 138ms/step - loss: 0.5840 - accuracy: 0.6890 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 7/20
313/313 [==============================] - 44s 140ms/step - loss: 0.5573 - accuracy: 0.7181 - val_loss: 0.6931 - val_ac

In [ ]:
x_test_texts = [" ".join([reverse_word_index.get(i - 3, '?') for i in review]) for review in x_test[:10000]]

In [ ]:
x_test_sequences = np.array(x_test_texts).reshape(-1)

In [ ]:
x_test_sequences.shape

(10000,)

In [ ]:
y_test[:10000].shape

(10000,)

In [ ]:
test_loss, test_acc = model.evaluate(x_test_sequences, y_test[:10000])
print(f"Test accuracy: {test_acc}")

313/313 [==============================] - 38s 123ms/step - loss: 0.6932 - accuracy: 0.4973
Test accuracy: 0.49729999899864197


Test accuracy did not perform well.  Starting over from beginning.

In [ ]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

word_index = imdb.get_word_index()
index_word = {i: word for word, i in word_index.items()}

In [ ]:
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=max_length,
    output_mode="int"
)

train_texts = [' '.join([index_word.get(i - 3, '?') for i in review]) for review in train_data]
text_vectorization.adapt(train_texts)

In [ ]:
path_to_glove_file = "glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

embedding_dim = 100
embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True
)

In [ ]:
decoded_test_data = [[index_word.get(i - 3, '?') for i in review] for review in test_data]
decoded_test_array = np.array([" ".join(review) for review in decoded_test_data])
vectorized_test = text_vectorization(decoded_test_array).numpy()

vectorized_train = text_vectorization(np.array([[s] for s in train_texts])).numpy()

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(
    vectorized_train,
    np.array(train_labels),
    epochs=20,
    batch_size=512,
    validation_split=0.2
)

test_loss, test_acc = model.evaluate(vectorized_test, np.array(test_labels))
print(f"Test accuracy: {test_acc}")

Epoch 1/20
40/40 [==============================] - 11s 88ms/step - loss: 0.7016 - accuracy: 0.5165 - val_loss: 0.6929 - val_accuracy: 0.5208
Epoch 2/20
40/40 [==============================] - 1s 29ms/step - loss: 0.6890 - accuracy: 0.5369 - val_loss: 0.6914 - val_accuracy: 0.5260
Epoch 3/20
40/40 [==============================] - 1s 29ms/step - loss: 0.6830 - accuracy: 0.5608 - val_loss: 0.6776 - val_accuracy: 0.5784
Epoch 4/20
40/40 [==============================] - 1s 27ms/step - loss: 0.6742 - accuracy: 0.5763 - val_loss: 0.7079 - val_accuracy: 0.5246
Epoch 5/20
40/40 [==============================] - 1s 26ms/step - loss: 0.6694 - accuracy: 0.5903 - val_loss: 0.6580 - val_accuracy: 0.6070
Epoch 6/20
40/40 [==============================] - 1s 26ms/step - loss: 0.6605 - accuracy: 0.6077 - val_loss: 0.6608 - val_accuracy: 0.6010
Epoch 7/20
40/40 [==============================] - 1s 26ms/step - loss: 0.6547 - accuracy: 0.6176 - val_loss: 0.6338 - val_accuracy: 0.6396
Epoch 8/20
4